In [9]:
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn import datasets
from sklearn import mixture
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.datasets import make_spd_matrix
plt.rcParams["axes.grid"] = False

In [10]:
import pandas as pd

In [11]:
n_samples = 10000

In [12]:
means = [[0 for i in range(32)],[1 for j in range(32)]]

mu, sigma = 0, 1 
s = np.random.normal(mu, sigma, 1)
means[1] = [s[0]*i for i in means[1]]


In [13]:
mean_datasets = []
c  = [0.5,1,1.5,2,3,4,8]
for i in c:
    temp = means.copy()
    temp[1] = [i*j for j in temp[1]]
    mean_datasets.append(temp)
for i in mean_datasets:
    print(i[1][:3])
print(len(mean_datasets))

[-0.3794119093224735, -0.3794119093224735, -0.3794119093224735]
[-0.758823818644947, -0.758823818644947, -0.758823818644947]
[-1.1382357279674205, -1.1382357279674205, -1.1382357279674205]
[-1.517647637289894, -1.517647637289894, -1.517647637289894]
[-2.276471455934841, -2.276471455934841, -2.276471455934841]
[-3.035295274579788, -3.035295274579788, -3.035295274579788]
[-6.070590549159576, -6.070590549159576, -6.070590549159576]
7


In [14]:
covs_datasets = []
for i in range(7):
    t = []
    for j in range(2):
        t.append(np.identity(32))
    covs_datasets.append(t)
print(len(covs_datasets))

7


In [15]:
sample_datasets = []
for i in range(7):
    dataset = []
    for mean, cov in zip(mean_datasets[i],covs_datasets[i]):
        x = np.random.multivariate_normal(mean, cov, n_samples)
        dataset += list(x)
    sample_datasets.append(np.array(dataset))

In [16]:
len(sample_datasets[0])

20000

In [17]:
k = 2

weights = np.ones((k)) / k
means = np.random.choice(sample_datasets[0].flatten(), (k,sample_datasets[0].shape[1]))
print(means.shape)
print(means[0])
print(weights)

(2, 32)
[-1.09679911  1.76221719  0.3415138   1.7270985  -0.24445627 -0.86308882
 -0.48841997  0.18815858 -2.6417089   0.10816897  2.25963239 -0.66151895
  0.1334857  -0.13675112  0.66090025 -1.59958322 -0.25607196 -2.01583511
  0.0136725  -0.29370427 -0.67964133 -0.45184314  0.9020419   0.72632128
  1.15078182  0.8449146  -1.64892721 -1.48742251 -0.28981104 -1.32659914
 -0.25723998 -1.58216594]
[0.5 0.5]


In [18]:
cov = []
for i in range(k):
    cov.append(make_spd_matrix(sample_datasets[0].shape[1]))
cov = np.array(cov)
print(cov.shape)
# a = multivariate_normal.pdf(x=sample_datasets[0], mean=means[1], cov=cov[1])
# print(a)


(2, 32, 32)


In [30]:
k = 2


for dataset in range(7):
    weights = np.ones((k)) / k
    means = np.random.choice(sample_datasets[dataset].flatten(), (k,sample_datasets[dataset].shape[1]))
#     print(means.shape)
#     print(means[0])
#     print(weights)

    cov = []
    for i in range(k):
        cov.append(make_spd_matrix(sample_datasets[dataset].shape[1]))
    cov = np.array(cov)
#     print(cov.shape)


    for iteration in range(50):    
        #expectation_step
        likelihood = []
        for j in range(k):
            t = multivariate_normal.pdf(x=sample_datasets[dataset], mean=means[j], cov=cov[j])
            likelihood.append(t)

        likelihood = np.array(likelihood)
        assert likelihood.shape == (k, len(sample_datasets[dataset]))
        b = []
        # Maximization step 

        for j in range(k):
            temp = (likelihood[j] * weights[j]) / (np.sum([likelihood[i] * weights[i] for i in range(k)], axis=0))
            b.append(temp)
            # update mean and variance
            means[j] = np.sum(b[j].reshape(len(sample_datasets[0]),1) * sample_datasets[dataset], axis=0) / (np.sum(b[j]))
            cov[j] = np.dot((b[j].reshape(len(sample_datasets[0]),1) * (sample_datasets[dataset] - means[j])).T, (sample_datasets[dataset] - means[j])) / (np.sum(b[j]))

            # update the weights        
            weights[j] = np.mean(b[j])

            assert cov.shape == (k, sample_datasets[0].shape[1], sample_datasets[0].shape[1])
            assert means.shape == (k, sample_datasets[0].shape[1])
    err = np.linalg.norm(means[0]-mean_datasets[dataset][0])+np.linalg.norm(means[1]-mean_datasets[dataset][1])
    print("error for dataset ", dataset," is ", err)

error for dataset  0  is  3.6963757383493743
error for dataset  1  is  5.88143561151477
error for dataset  2  is  12.871088368253506
error for dataset  3  is  0.11246857055503309
error for dataset  4  is  25.734877856399464


/home/rohit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
/home/rohit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


ValueError: array must not contain infs or NaNs

# Kmeans++

In [21]:
import sys
def distance(p1, p2): 
    return np.sum((p1 - p2)**2) 
   
# initialisation algorithm 
def initialize(data, k): 

    centroids = [] 
    centroids.append(data[np.random.randint( 
            data.shape[0]), :]) 
    
    for c_id in range(k - 1):          
        dist = [] 
        for i in range(data.shape[0]): 
            point = data[i, :] 
            d = sys.maxsize 

            for j in range(len(centroids)): 
                temp_dist = distance(point, centroids[j]) 
                d = min(d, temp_dist) 
            dist.append(d) 
              
        dist = np.array(dist) 
        next_centroid = data[np.argmax(dist), :] 
        centroids.append(next_centroid) 
        dist = [] 
    return centroids 
   
# call the initialize function to get the centroids 


In [22]:
def assign_clusters(data,centers,k):
    clusters = [[] for i in range(k)]
    for i in data:
        distances = []
        for j in range(k):
            distances.append(distance(i,centers[j]))
        clusters[distances.index(min(distances))].append(i)
    return clusters
def recalculate_centres(clusters):
    centres = []
    for i in clusters:
        centres.append(np.mean(np.array(i)))
    return centres    
def kmeans(data,initial_centers,iterations,k):
    centers = initial_centers.copy()
#     centers = np.random.rand(4,32)
    for i in range(iterations):
        clusters = assign_clusters(data,centers,k)
        centers_new = recalculate_centres(clusters)
        centers = centers_new
    return centers

centroids = initialize(sample_datasets[0], k = 2)
new_centers = kmeans(sample_datasets[0],centroids,4,2)
err = np.linalg.norm(new_centers[0]-mean_datasets[0][0])+np.linalg.norm(new_centers[1]-mean_datasets[0][1])
print(err)

0.2946844594789496


In [23]:
for dataset in range(7):
    centroids = initialize(sample_datasets[dataset], k = 2)
    new_centers = kmeans(sample_datasets[dataset],centroids,4,2)
    err = np.linalg.norm(new_centers[0]-mean_datasets[dataset][0])+np.linalg.norm(new_centers[1]-mean_datasets[dataset][1])
    print("error for dataset ", dataset," is ", err)

error for dataset  0  is  4.585304483683002
error for dataset  1  is  0.05096630793423345
error for dataset  2  is  0.010523525178146628
error for dataset  3  is  0.010666201380545309
error for dataset  4  is  0.020802875302949376
error for dataset  5  is  34.354062420129424
error for dataset  6  is  0.0030887618573066777


In [25]:
_33

NameError: name '_33' is not defined